In [83]:
!pip install git+https://github.com/pytube/pytube

  Cloning https://github.com/pytube/pytube to /private/var/folders/wm/vt5jkwy10xs_7zv7nc5w67wr0000gn/T/pip-req-build-lu6r484l
  Running command git clone --filter=blob:none --quiet https://github.com/pytube/pytube /private/var/folders/wm/vt5jkwy10xs_7zv7nc5w67wr0000gn/T/pip-req-build-lu6r484l
  Resolved https://github.com/pytube/pytube to commit a32fff39058a6f7e5e59ecd06a7467b71197ce35
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [85]:
from dotenv import load_dotenv
import os
import openai

# Load environment variables
load_dotenv(dotenv_path="../env_file/.env")

# Set API key for OpenAI client
openai.api_key = os.getenv("OPENAI_API_KEY")

In [87]:
import yt_dlp

def download_audio(video_url, output_path="audio.mp3"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_path,
        'ffmpeg_location': '/opt/homebrew/bin',  # This line tells yt-dlp where to find ffmpeg
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        return output_path
    except Exception as e:
        print(f"Error downloading audio: {e}")
        return None

In [89]:
def transcribe_audio(audio_path):
    try:
        with open(audio_path, "rb") as audio_file:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                response_format="text"
            )
        return transcript
    except Exception as e:
        print(f"Error transcribing audio: {e}")
        return None

In [91]:
def build_prompt(transcript_text, mode="basic"):
    if mode == "bullets":
        return f"Summarize the following transcript into concise bullet points with timestamps when possible:\n\n{transcript_text}"
    elif mode == "quotes":
        return f"Extract the most insightful or notable quotes with timestamps from the following transcript:\n\n{transcript_text}"
    elif mode == "insights":
        return f"Summarize the following transcript into key insights, trends, or warnings with timestamp references:\n\n{transcript_text}"
    else:
        return f"Summarize the following transcript:\n\n{transcript_text}"

In [93]:
def summarize_transcript(transcript_text, mode="basic"):
    try:
        prompt = build_prompt(transcript_text, mode)
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=800
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error summarizing transcript: {e}")
        return None

In [104]:
url = input("Enter YouTube URL: ")
mode = input("Choose summary style (basic, bullets, quotes, insights): ")
audio_file = download_audio(url)

if audio_file:
    transcript_text = transcribe_audio(audio_file)
    if transcript_text:
        print("\n--- Transcript fetched ---\n")
        summary = summarize_transcript(transcript_text, mode)
        if summary:
            print("\n--- Summary ---\n")
            print(summary)
        else:
            print("Summary generation failed.")
    else:
        print("Transcription failed.")
else:
    print("Audio download failed.")

Enter YouTube URL:  https://www.youtube.com/watch?v=59t62NHJFH4
Choose summary style (basic, bullets, quotes, insights):  bullets


                             
--- Transcript fetched ---


--- Summary ---

- AI is rapidly progressing with significant developments such as generative AI and reasoning models. 
- In 2025, a Chinese startup, DeepSeek, introduced an AI model that disrupted the industry, causing a trillion-dollar valuation drop in major AI players like OpenAI and Microsoft.
- AI is impacting many areas including analytics, machine learning, pattern recognition, content creation, and problem-solving.
- The cutting edge of AI is moving towards agentic AI, where AI acts as agents performing tasks such as booking flights or restocking fridges.
- With AI being able to think, reason, and make decisions, there is a potential for misuse, misinformation, and influence.
- Interaction with AI is mostly through prompting, but there are limitations such as the hallucination problem where AI provides incorrect information when it doesn't know an answer.
- AI systems are programmed with personalities designed to be he